In [3]:
# base imports
import pandas as pd
import numpy as np

In [16]:
# Column met_o had values 1/2. It needs to be changed to 0/1.
df = pd.read_csv('./data_for_matrix.csv')
df['met_o'] = df['met_o'].apply(lambda x: x - 1)
df.to_csv('data_for_matrix.csv', index=False)

### Below you can see code that prepares data for the basic matrix factorization.
Here in the base matrix we only have information about match. So we need a set of vectors, where each vector describes each date (holds both ids and information about match).

Task will requite two base matrices.
1. Matrix where men are "users" and women are "products". It will then be used to recommend women to men because matrix will say what's the predicted rating of a woman in eyes of man. Basically it will answer the question: **"How likely is that a man will like a woman?"**. Let's call this matrix/data frame **"men_like_women"**.
1. Matrix where women are "users" and men are "products". It will then be used to recommend men to women because matrix will say what's the predicted rating of a man in eyes of woman. Basically it will answer the question: **"How likely is that a woman will like a man?"**. Let's call this matrix/data frame **"women_like_men"**.

Why such analogies? It may help to understand how do this human relations task translates into recommender systems world.

These matrices will be used to train models (with matrix factorization) which will then be saved into csv files.

In [21]:
# Split into vectors, let's have two matrices as describes above.
base_df = pd.read_csv('./data_for_matrix.csv')
men_like_women_data = []
women_like_men_data = []

for _, row in base_df.iterrows():
    vector = {
        'id': row['iid'],
        'pid': row['pid'],
        'match': row['match'],
    }
    if row['gender'] == 0:
        women_like_men_data.append(vector)
    else:
        men_like_women_data.append(vector)

men_like_women_df = pd.DataFrame(men_like_women_data)
women_like_men_df = pd.DataFrame(women_like_men_data)

print("men_like_women_df:")
print(men_like_women_df)
print("\nwomen_like_men_df:")
print(women_like_men_df)

men_like_women_df:
         id    pid  match
0      11.0    1.0    0.0
1      11.0    2.0    0.0
2      11.0    3.0    0.0
3      11.0    4.0    0.0
4      11.0    5.0    0.0
...     ...    ...    ...
4179  552.0  526.0    0.0
4180  552.0  527.0    0.0
4181  552.0  528.0    0.0
4182  552.0  529.0    0.0
4183  552.0  530.0    0.0

[4184 rows x 3 columns]

women_like_men_df:
         id    pid  match
0       1.0   11.0    0.0
1       1.0   12.0    0.0
2       1.0   13.0    1.0
3       1.0   14.0    1.0
4       1.0   15.0    1.0
...     ...    ...    ...
4179  530.0  548.0    0.0
4180  530.0  549.0    0.0
4181  530.0  550.0    0.0
4182  530.0  551.0    0.0
4183  530.0  552.0    0.0

[4184 rows x 3 columns]


### Let's prepare train and test data sets
Standard sklearn function train_test_split doesn't do a job here because both test and train data sets should include all people. So after using train_test_split we need to transition some people between sets to ensure that both sets are correct. The same goes for partners.

In [42]:
from sklearn.model_selection import train_test_split as tts

def train_test_split(df, test_size=0.2):
    # Use standard train_test_split.
    df_train, df_test = tts(df, test_size=test_size)
    no_train_unique = len(df_train.id.unique())
    no_test_unique = len(df_test.id.unique())
    
    # See what people are missing in the test set.
    diff = np.setdiff1d(df_train.id.unique(), df_test.id.unique())
    for id in diff:
        # For every missing person we need to exchange them for a
        # different one so that sets still have the same number of elements.
        person_to_send_to_test = df_train.loc[df_train.id == id].iloc[0]
        person_to_send_to_train = None
        # Choose some person from the test set to send to the train set.
        ids = df_test.id.unique()
        np.random.shuffle(ids)
        for test_id in ids:
            person = df_test.loc[df_test.id == test_id]
            if len(person.index) > 1:
                person_to_send_to_train = person.iloc[0]
                break
        if person_to_send_to_train is not None:
            # Remove people that transit from old sets.
            # .name holds the id of that row in the oryginal df.
            df_train = df_train.drop(person_to_send_to_test.name)
            df_test = df_test.drop(person_to_send_to_train.name)
            # Add new people to sets.
            df_train = pd.concat([df_train, person_to_send_to_train.to_frame().T], ignore_index=True)
            df_test = pd.concat([df_test, person_to_send_to_test.to_frame().T], ignore_index=True)
        else:
            raise Exception("Couldn't find any person from people to send from the test to the train.")
        
    # See what partners are missing in the test set.
    diff = np.setdiff1d(df_train.pid.unique(), df_test.pid.unique())
    for pid in diff:
        # For every missing partner we need to exchange them for a
        # different one so that sets still have the same number of elements.
        partner_to_send_to_test = df_train.loc[df_train.pid == pid].iloc[0]
        partner_to_send_to_train = None
        
        # Choose some partner from the test set to send to the train set.
        # Need to make sure that both sets will still have all the people.
        pids = df_test.pid.unique()
        np.random.shuffle(pids)
        for test_pid in pids:
            partner = df_test.loc[df_test.pid == test_pid]
            if len(partner.index) > 1:
                # Make sure we don't remove a person completely.
                id = partner.iloc[0].id
                person_qty = len(df_test.loc[df_test.id == id].index)
                if person_qty > 1:
                    partner_to_send_to_train = partner.iloc[0]
                    break
        if partner_to_send_to_train is not None:
            # Remove partners that transit from old sets.
            df_train = df_train.drop(partner_to_send_to_test.name)
            df_test = df_test.drop(partner_to_send_to_train.name)
            # Add new people to sets.
            df_train = pd.concat([df_train, partner_to_send_to_train.to_frame().T], ignore_index=True)
            df_test = pd.concat([df_test, partner_to_send_to_test.to_frame().T], ignore_index=True)
        else:
            raise Exception("Couldn't find any partner from partners to send from the test to the train.")
                    
    df_train = df_train.sort_values(by='id')
    df_test = df_test.sort_values(by='id')
    return df_train.reset_index(drop=True), df_test.sort_values(by='id').reset_index(drop=True)


men_like_women_train_df, men_like_women_test_df = train_test_split(men_like_women_df, test_size=0.2)
men_like_women_no_men = len(men_like_women_train_df.id.unique())
men_like_women_no_women = len(men_like_women_train_df.pid.unique())

women_like_men_train_df, women_like_men_test_df = train_test_split(women_like_men_df, test_size=0.2)
women_like_men_no_women = len(women_like_men_train_df.id.unique())
women_like_men_no_men = len(women_like_men_train_df.pid.unique())

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dev = torch.device('cpu')

## Let's create matrix factorization models

We will create and train 4 base MF (matrix factorization) models:
1. without bias and uniform (0, 0.2) weight initialization,
2. without bias and xavier initialization,
3. with bias and uniform (0, 0.2) weight initialization,
4. with bias and xavier initialization,

For each model will do a cross validation to learn the best hyperparameters and then we will compare the results and choose the best model.

Some general explanations for models:
* Models are train on only one batch because our data set is rather small.

Good reading resource: https://towardsdatascience.com/weight-initialization-techniques-in-neural-networks-26c649eb3b78

### First matrix factorization without bias

In [6]:
class MatrixFactorizationWithoutBiasNoXavier(nn.Module):
    def __init__(self, num_people, num_partners, emb_size=100):
        super(MatrixFactorizationWithoutBiasNoXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.person_emb.weight.data.uniform_(0,0.2)
        self.partner_emb.weight.data.uniform_(0,0.2)
        
    def forward(self, u, v):
        print(u)
        print(u.size())
        print(self.person_emb)
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1))
    
    
class MatrixFactorizationWithoutBiasXavier(nn.Module):
    def __init__(self, num_people, num_partners, emb_size=100):
        super(MatrixFactorizationWithoutBiasXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        torch.nn.init.xavier_uniform_(self.person_emb.weight)
        torch.nn.init.xavier_uniform_(self.partner_emb.weight)
        
    def forward(self, u, v):
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1))

    
# Example small models demonstrating weights
example_model_no_xavier = MatrixFactorizationWithoutBiasNoXavier(10, 10, 3)
example_model_xavier = MatrixFactorizationWithoutBiasXavier(10, 10, 3)
print("Model with without xavier weights are:\n")
for p in example_model_no_xavier.parameters():
    print(p)
print('\n\n', '='*20)
print("\n\nModel with with xavier weights are:\n")
for p in example_model_xavier.parameters():
    print(p)

Model with without xavier weights are:

Parameter containing:
tensor([[0.1244, 0.1307, 0.0249],
        [0.0005, 0.0587, 0.1041],
        [0.0164, 0.1728, 0.1264],
        [0.1940, 0.0844, 0.1358],
        [0.0625, 0.0590, 0.0428],
        [0.0577, 0.0820, 0.1289],
        [0.0299, 0.1766, 0.1377],
        [0.0213, 0.0361, 0.0297],
        [0.1479, 0.0010, 0.0518],
        [0.0188, 0.1637, 0.1444]], requires_grad=True)
Parameter containing:
tensor([[0.1285, 0.1068, 0.0154],
        [0.0566, 0.1849, 0.1884],
        [0.1578, 0.0704, 0.0938],
        [0.1584, 0.0837, 0.0974],
        [0.0084, 0.1098, 0.0266],
        [0.1688, 0.1899, 0.0341],
        [0.1609, 0.1743, 0.1368],
        [0.1531, 0.1416, 0.1549],
        [0.1189, 0.0629, 0.0257],
        [0.0658, 0.0354, 0.1093]], requires_grad=True)




Model with with xavier weights are:

Parameter containing:
tensor([[-0.3315,  0.1792,  0.0646],
        [-0.2165, -0.3660, -0.1811],
        [-0.5668,  0.5622, -0.3133],
        [-0.5317,  0

### First matrix factorization without bias

In [7]:
class MatrixFactorizationWithBiasNoXavier(nn.Module):
    def __init__(self, num_people, num_partners, emb_size=100):
        super(MatrixFactorizationWithBiasNoXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.person_bias = nn.Embedding(num_people, 1)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.parnter_bias = nn.Embedding(num_partners, 1)
        torch.nn.init.xavier_uniform_(self.person_emb.weight)
        torch.nn.init.xavier_uniform_(self.partner_emb.weight)
        self.person_bias.weight.data.uniform_(-0.01,0.01)
        self.parnter_bias.weight.data.uniform_(-0.01,0.01)
            
    def forward(self, u, v):
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1))
    
    
class MatrixFactorizationWithBiasNoXavier(nn.Module):
    def __init__(self, num_people, num_partners, emb_size=100):
        super(MatrixFactorizationWithBiasNoXavier, self).__init__()
        self.person_emb = nn.Embedding(num_people, emb_size)
        self.person_bias = nn.Embedding(num_people, 1)
        self.partner_emb = nn.Embedding(num_partners, emb_size)
        self.parnter_bias = nn.Embedding(num_partners, 1)
        self.person_emb.weight.data.uniform_(0,0.2)
        self.partner_emb.weight.data.uniform_(0,0.2)
        self.person_bias.weight.data.uniform_(-0.01,0.01)
        self.parnter_bias.weight.data.uniform_(-0.01,0.01)
            
    def forward(self, u, v):
        u = self.person_emb(u)
        v = self.partner_emb(v)
        # calculate dot product
        # u*v is a element wise vector multiplication
        return torch.sigmoid((u*v).sum(1))
    

# Example small models demonstrating weights
example_model_no_xavier = MatrixFactorizationWithBiasNoXavier(10, 10, 3)
example_model_xavier = MatrixFactorizationWithBiasNoXavier(10, 10, 3)
print("Model with without xavier weights are:\n")
for p in example_model_no_xavier.parameters():
    print(p)
print('\n\n', '='*20)
print("\n\nModel with with xavier weights are:\n")
for p in example_model_xavier.parameters():
    print(p)

Model with without xavier weights are:

Parameter containing:
tensor([[0.1928, 0.1053, 0.0974],
        [0.1194, 0.1117, 0.0583],
        [0.1270, 0.0447, 0.0269],
        [0.0896, 0.0745, 0.1594],
        [0.0386, 0.0972, 0.0510],
        [0.1332, 0.0608, 0.0569],
        [0.0951, 0.1882, 0.1080],
        [0.1219, 0.1552, 0.1688],
        [0.0434, 0.0817, 0.0974],
        [0.0399, 0.1130, 0.1127]], requires_grad=True)
Parameter containing:
tensor([[-2.3825e-03],
        [-5.8028e-03],
        [ 7.0894e-05],
        [ 9.8498e-03],
        [-1.2397e-03],
        [-1.6610e-03],
        [ 1.3798e-03],
        [ 3.7889e-04],
        [-7.3799e-03],
        [-6.1363e-03]], requires_grad=True)
Parameter containing:
tensor([[0.0266, 0.0960, 0.1000],
        [0.1651, 0.1408, 0.0188],
        [0.1378, 0.0805, 0.1701],
        [0.1724, 0.1328, 0.0855],
        [0.1405, 0.0435, 0.0099],
        [0.0347, 0.0498, 0.0384],
        [0.0520, 0.1570, 0.1217],
        [0.0743, 0.1995, 0.0518],
        [0

### Training and testing functions are below

In [43]:
def test(model, df_test):
    model.eval()
    # .to(dev) puts code on either gpu or cpu.
    people = torch.LongTensor(df_val.userId.values).to(dev)
    partners = torch.LongTensor(df_val.movieId.values).to(dev)
    attraction = torch.FloatTensor(df_val.rating.values).to(dev)
    y_hat = model(people, partners)
    loss = F.mse_loss(y_hat, attraction)
    print('test loss %.3f ' % loss.item())


# Default values assigned below are ones that I found online.
# Cross validadtion will be done later but it's good to have some defaults.
def train(model, df_train, epochs=100, learning_rate=0.01, weight_decay=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    model.train()
    
    for epoch in range(epochs):
        # From numpy to PyTorch tensors.
        # .to(dev) puts code on either gpu or cpu.
        people = torch.LongTensor(df_train.id.values).to(dev)
        partners = torch.LongTensor(df_train.pid.values).to(dev)
        attraction = torch.FloatTensor(df_train.match.values).to(dev)
        
        # calls forward method of the model
        y_hat = model(people, partners)
        # Using mean squared errors loss function
        loss = F.mse_loss(y_hat, attraction)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % 10: 
            print(loss.item())

## Let's finally train models and choose the best one

### Let's first train men_like_women

In [44]:
model_no_bias_no_xavier = MatrixFactorizationWithoutBiasNoXavier(men_like_women_no_men, men_like_women_no_women).to(dev)

train(model_no_bias_no_xavier, men_like_women_train_df, weight_decay=0.01)
test(model_no_bias_no_xavier, men_like_women_test_df)

tensor([ 11,  11,  11,  ..., 552, 552, 552])
torch.Size([3347])
Embedding(277, 100)


IndexError: index out of range in self